In [4]:
import pandas as pd
import numpy as np
import os
import keras
import sklearn
import json
import requests
import itertools
import datetime
import math

import seaborn as sns

from scipy import linalg
from scipy.spatial.distance import pdist

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import OneHotEncoder

from streetaddress import StreetAddressFormatter, StreetAddressParser

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [ ]:
datadir = '../data/plows/v1/clean/'
file_name = 'all_plows.csv'
data_file = os.path.join(datadir, file_name)

In [90]:
high_level_file_name = 'high_level_plows.csv'
high_level_data_file = os.path.join(datadir, high_level_file_name)

In [47]:
df = pd.read_csv(data_file).fillna('')
df.head()

,activity_type,address,latitude,longitude,repair_type,strHeading,truck_name,time,date,street,street_num
0,Stop Moving,306 SENECA TNPK E TO ARSENAL ...,42.999824,-76.138107,SNOW & ICE,E,178.0,00:00:07,01/08/2018,SENECA TNPK E TO ARSENAL,306
1,,106 ARSENAL DR ...,42.999739,-76.138110,SNOW & ICE,E,178.0,00:01:07,01/08/2018,ARSENAL DR,106
2,,306 SENECA TNPK E TO ARSENAL ...,42.999786,-76.138090,SNOW & ICE,E,178.0,00:02:07,01/08/2018,SENECA TNPK E TO ARSENAL,306
3,,121 ARSENAL DR ...,42.999810,-76.137820,SNOW & ICE,E,178.0,00:03:07,01/08/2018,ARSENAL DR,121
4,Start Moving,121 ARSENAL DR ...,42.999808,-76.137770,SNOW & ICE,E,178.0,00:03:09,01/08/2018,ARSENAL DR,121


In [56]:
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])

In [58]:
df['epoch_time'] = df['datetime'].map(lambda x: x.timestamp())

In [ ]:
plow_passes_groups = df.groupby(['truck_name', 'date', 'street'])
plow_passes = plow_passes_groups.agg({'epoch_time' : [np.min, np.max]}).reset_index()
plow_passes['time_elapsed'] = plow_passes['epoch_time']['amax'] - plow_passes['epoch_time']['amin']

In [77]:
plow_passes.to_csv(index=False)

In [89]:
plow_passes[plow_passes['time_elapsed'] != 0.0]

truck_name        date                   street    epoch_time  \
                                                               amin   
2           73.0  01/01/2018            BALLANTYNE RD  1.514773e+09   
5           73.0  01/01/2018              BELLEVUE ST  1.514769e+09   
7           73.0  01/01/2018              BRIGHTON ST  1.514777e+09   
8           73.0  01/01/2018               BURNET AVE  1.514783e+09   
9           73.0  01/01/2018                 CANAL ST  1.514765e+09   
14          73.0  01/01/2018               DORWIN AVE  1.514768e+09   
18          73.0  01/01/2018  EB I 690 SYRACUSE 13210  1.514765e+09   
20          73.0  01/01/2018                  EDNA RD  1.514767e+09   
21          73.0  01/01/2018                 ERIE AVE  1.514765e+09   
22          73.0  01/01/2018                ERIE BLVD  1.514778e+09   
23          73.0  01/01/2018                  ERIE ST  1.514765e+09   
24          73.0  01/01/2018              FAYETTE AVE  1.514778e+09   
25          73.0  01/01/2018               FAYETTE ST  1.514777e+09   
26          73.0  01/01/2018             FILLMORE AVE  1.514770e+09   
32          73.0  01/01/2018               KENNEDY ST  1.514775e+09   
34          73.0  01/01/2018             KIRK PARK DR  1.514773e+09   
37          73.0  01/01/2018                 MENLO RD  1.514773e+09   
38          73.0  01/01/2018              MIDLAND AVE  1.514768e+09   
39          73.0  01/01/2018               MIDLAND ST  1.514769e+09   
40          73.0  01/01/2018            MONTGOMERY ST  1.514766e+09   
47          73.0  01/01/2018                  PEAT ST  1.514765e+09   
50          73.0  01/01/2018                  RICH ST  1.514769e+09   
52          73.0  01/01/2018               SALINA AVE  1.514767e+09   
54          73.0  01/01/2018                SALINA ST  1.514766e+09   
55          73.0  01/01/2018            SENECA TNPK W  1.514768e+09   
58          73.0  01/01/2018                SOUTH AVE  1.514775e+09   
59          73.0  01/01/2018                 SOUTH ST  1.514769e+09   
60          73.0  01/01/2018                STATE AVE  1.514777e+09   
61          73.0  01/01/2018                 STATE ST  1.514766e+09   
62          73.0  01/01/2018              STERLING ST  1.514769e+09   
...          ...         ...                      ...           ...   
24735      285.0  01/06/2018               MILFORD CT  1.515201e+09   
24736      285.0  01/06/2018               MILFORD DR  1.515200e+09   
24740      285.0  01/06/2018               MOSLEY AVE  1.515203e+09   
24741      285.0  01/06/2018                MOSLEY DR  1.515199e+09   
24742      285.0  01/06/2018                MOSLEY RD  1.515199e+09   
24743      285.0  01/06/2018              NICHOLS AVE  1.515199e+09   
24744      285.0  01/06/2018               NICHOLS RD  1.515199e+09   
24745      285.0  01/06/2018            NORTHCLIFF RD  1.515201e+09   
24747      285.0  01/06/2018            ONONDAGA BLVD  1.515219e+09   
24748      285.0  01/06/2018              ONONDAGA ST  1.515217e+09   
24749      285.0  01/06/2018                  PEAT ST  1.515198e+09   
24750      285.0  01/06/2018             PLYMOUTH AVE  1.515203e+09   
24751      285.0  01/06/2018              PLYMOUTH DR  1.515200e+09   
24752      285.0  01/06/2018              PLYMOUTH RD  1.515202e+09   
24753      285.0  01/06/2018                 REED AVE  1.515219e+09   
24754      285.0  01/06/2018            RIDGEWOOD AVE  1.515207e+09   
24755      285.0  01/06/2018             RIDGEWOOD DR  1.515202e+09   
24756      285.0  01/06/2018             RIDGEWOOD RD  1.515205e+09   
24758      285.0  01/06/2018               ROXBURY RD  1.515205e+09   
24759      285.0  01/06/2018             STAFFORD AVE  1.515198e+09   
24760      285.0  01/06/2018                 STATE ST  1.515217e+09   
24761      285.0  01/06/2018                STOLP AVE  1.515218e+09   
24762      285.0  01/06/2018                 STOLP RD  1.515219e+09   
24763      285.0  01/06/2018  

In [12]:
type(df['time'][0])


str